<a href="https://colab.research.google.com/github/Maziger/Laksegate-master-thesis/blob/main/POC/experiment_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata

user = "Maziger"
repo = "Laksegate-master-thesis"

# remove local directory if it already exists
if os.path.isdir(repo):
    !rm -rf {repo}

!git clone https://github.com/{user}/{repo}.git
%cd Laksegate-master-thesis/


Cloning into 'Laksegate-master-thesis'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 1.08 MiB | 6.98 MiB/s, done.


In [15]:
import pandas as pd
path = "POC/Forecasts_DE_DNN_LEAR_ensembles.csv"
data = pd.read_csv(path)
data = data[['Unnamed: 0', 'Real price']]
train = data[:14000]
test = data[14000:]